In [7]:
from typing import List

import numpy as np

from nn.value import Value
from nn.models import Perceptron
from nn.optimizer import AdamOptimizer

In [8]:
def make_perceptron_value(data: List[Value], weights: List[Value], bias: Value) -> Value:
    """Create and return a perceptron

    Parameters
    ----------
    data: list of Value objects (len should be >= 1)
        each entry represents a feature of x
    weights: list of Value objects (len should be >= 1)
        list of weights
    bias: Value
        represents a bias term
    """
    summands = map(lambda d, w: d * w, data, weights)
    
    linear_comb = next(summands)
    for summand in summands:
        linear_comb = linear_comb + summand

    logit = linear_comb + bias
    
    probability = logit.sigmoid()

    return probability

def make_binary_crossentropy_loss(prediction_values: List[Value], ground_truth: List[float]):
    loss = 0
    for p, g in zip(prediction_values, ground_truth):
        if np.isclose(g, 0):
            example_cost = (1 - p).log()
        elif np.isclose(g, 1):
            example_cost = p.log()
        loss = loss + example_cost
    loss = (-1) * loss
    return loss

This is an example that demonstrates how to creates loss function:

In [9]:
weights = [Value(0), Value(1)]
bias = Value(2)

data1 = [Value(3), Value(4)]
perceptron1 = make_perceptron_value(data1, weights, bias)

data2 = [Value(0), Value(0)]
perceptron2 = make_perceptron_value(data2, weights, bias)

loss = make_binary_crossentropy_loss([perceptron1, perceptron2], [1, 0])

This example demonstrates how to train a perceptron to behave like an AND gate:

In [19]:
X = [
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
    ]

Y = [0, 0, 0, 1]

perceptron = Perceptron(no_weights=2)
adam_optimizer = AdamOptimizer(perceptron.parameters(), lr=0.5)

def simple_gradient_descent_iteration(model, X, Y, optimizer):
    optimizer.zero_grad()

    loss = sum((model(x) - y)**2 for x, y in zip(X, Y))

    loss.back_prop()
    print("Loss =", loss.data)

    optimizer.step()

num_iterations = 100

for i in range(num_iterations):
    print(f"Iteration {i}")
    simple_gradient_descent_iteration(perceptron, X, Y, adam_optimizer)
    print("y_pred =", *[perceptron(x).data for x in X])

Iteration 0
Loss = 1.4031262483018914
y_pred = 0.5060963145098841 0.5742552491057735 0.32750498616094864 0.39063498081802217
Iteration 1
Loss = 1.0644878132489568
y_pred = 0.3904820252921102 0.3812956197707211 0.2287006163201641 0.22193441499000968
Iteration 2
Loss = 0.9555525882147278
y_pred = 0.29588224183183 0.27515864055193157 0.1896866109614129 0.17455702248244423
Iteration 3
Loss = 0.8805956980128911
y_pred = 0.22619109822132777 0.22961847902527815 0.1772472236144697 0.1801055930470825
Iteration 4
Loss = 0.8075304756561665
y_pred = 0.17844149234726883 0.21988184942758937 0.18337606779600776 0.225647653011779
Iteration 5
Loss = 0.7134377334253097
y_pred = 0.14744288341323125 0.23609711600376232 0.2074285012336721 0.3186692556712497
Iteration 6
Loss = 0.5847194183461181
y_pred = 0.1277064643220684 0.27491006629317266 0.25049504104059167 0.4639541705603528
Iteration 7
Loss = 0.44197738242452733
y_pred = 0.11322338315385522 0.3292261364573901 0.30795155588668893 0.6310746218406588
It